# Prise en main de Polars

`Polars` est un package `Python` permettant de manipuler les données
tabulaires à partir de différents types de fichiers (CSV, Parquet,
etc.). Il est une alternative directe et moderne à `Pandas`, pensée pour
être très performante tout en offrant une syntaxe compréhensible à pour
des *data scientists* habitués à d’autres *frameworks* de manipulation
de données comme `dplyr`.

Ce notebook offre un complément à
l’[article](https://ssphub.netlify.app/post/polars/) publié sur le blog
du réseau des data scientists de la statistique publique. Les exemples
sont reproductibles dans de nombreux environnements, à condition
d’installer les packages comme indiqué ci-dessous. Les utilisateurs du
SSP Cloud ou de Colab pourront directement ouvrir ce notebook en
utilisant les boutons suivants:

<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&amp;onyxia.friendlyName=%C2%ABpython-datascience%C2%BB&amp;init.personalInit=%C2%ABhttps%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmaster%2Fsspcloud%2Finit-jupyter.sh%C2%BB&amp;init.personalInitArgs=%C2%ABmanipulation%2002b_pandas_TP%C2%BB&amp;security.allowlist.enabled=false" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSPcloud-Tester%20via%20SSP--cloud-informational&amp;color=yellow?logo=Python" alt="Onyxia"></a>
<a href="http://colab.research.google.com/github/romaintailhurat/miscbooks/blob/main/polars-tuto.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

Les exemples fournis dans ce notebook utiliseront les données de la
[base permanente des équipements
(BPE)](https://www.insee.fr/fr/metadonnees/source/serie/s1161).

Pour faire le parallèle avec les exemples pour [découvrir le `tidyverse`
dans
`utilitr`](https://www.book.utilitr.org/03_fiches_thematiques/fiche_tidyverse)),
ce *notebook* exploite la [Base Permanente des Equipements
(BPE)](https://www.insee.fr/fr/metadonnees/source/serie/s1161).

In [1]:
import os
import polars as pl
import s3fs
from pynsee.download import download_file

<div class="alert alert-danger" role="alert">
<i class="fa-solid fa-triangle-exclamation"></i> Warning</h3>

Outre `Polars`, ce notebook propose des compléments qui utilisent les
packages [`s3fs`](https://s3fs.readthedocs.io/en/latest/) et
[`Pynsee`](https://github.com/InseeFrLab/pynsee) pour, respectivement,
interagir avec un espace de stockage distant de type `S3` et récupérer
les données d’illustration sur le site
[insee.fr](https://www.insee.fr/fr/accueil). `Pynsee` est nécessaire en
début de *notebook* pour faciliter la récupération de la source de
données utilisée. `s3fs` n’est nécessaire que pour faire tourner les
exemples présents dans les encadrés proposant des détails
supplémentaires ou alternatifs.

``` python
!pip install polars pynsee[full] s3fs
```

</div>

# Lecture de données

A partir d’un csv, il est possible de créer un `DataFrame` `Polars` de
plusieurs manières :

1.  Charger les données via `Pandas` puis les transformer en objet
    `Polars`
2.  Charger les données directement avec `Polars` via une fonction
    `read_csv` qui fonctionne, en apparence, de la même manière que
    celle de `Pandas`

Ces deux approches sont présentées dans les prochaines parties.

## A partir d’un objet `Pandas`

Le *package* `Pynsee` fonctionne en deux temps pour créer un `DataFrame`
`Pandas` à partir des données disponibles sur le site
[insee.fr](https://www.insee.fr/fr/accueil) :

1.  Récupération des données depuis le site
    [insee.fr](https://www.insee.fr/fr/accueil) à partir des paramètres
    de la fonction `download_file` (source, année…)
2.  Import sous forme de `DataFrame` `Pandas` avec un typage pré-défini
    dans le *package*, adapté à la source

Pour récupérer des données via `Pynsee`, la fonction de référence est
`download_file`:

In [3]:
pandas_df_bpe = download_file("BPE_ENS")

`Pynsee` renvoie un `DataFrame` `Pandas` correspondant à la source
désiré :

In [4]:
pandas_df_bpe.__class__

pandas.core.frame.DataFrame

<div class="alert alert-danger" role="alert">
<i class="fa-solid fa-triangle-exclamation"></i> Warning</h3>

Ces deux étapes sont là pour éviter le téléchargement manuel du CSV.
Elles pourraient être remplacées par un `pd.read_csv` vers un URL bien
choisi. Cependant le typage des données pourrait ne pas être optimal.

</div>

La conversion d’un objet `Pandas` en `Polars` se fait via la méthode
`from_pandas`:

In [5]:
df = pl.from_pandas(pandas_df_bpe)

Les `DataFrame` `Polars` apparaissent de manière différente des
`DataFrame` `Pandas` dans la console ou dans le *display* de `Jupyter`:

-   `pd.DataFrame`:

In [6]:
pandas_df_bpe.head(2)

,AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
0,524,2021,01093,01,01001,A,200069193,010010000,84,A1,A129,CSZ,1
1,524,2021,01093,01,01001,A,200069193,010010000,84,A4,A401,CSZ,2


-   `pl.DataFrame`:

In [7]:
df.head(2)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""


On va écrire au format
[Parquet](https://pythonds.linogaliana.fr/reads3/) ce `DataFrame` pour
ensuite illustrer la lecture à partir de ce format, plus performant.

In [8]:
df.write_parquet("bpe.parquet")

## En lecture directe depuis un CSV

Pour faciliter l’usage des données de la [Base Permanente des
Equipements
(BPE)](https://www.insee.fr/fr/metadonnees/source/serie/s1161), une
copie de la version 2019 disponible sur le site de l’Insee est mise à
disposition, prête à l’emploi. Elle est disponible depuis l’URL :
https://minio.lab.sspcloud.fr/donnees-insee/diffusion/BPE/2019/BPE_ENS.csv

In [9]:
url = "https://minio.lab.sspcloud.fr/donnees-insee/diffusion/BPE/2019/BPE_ENS.csv"
df_bpe = pl.read_csv(
    url, sep = ";",
    dtypes={
        "DEP": pl.Categorical,
        "DEPCOM": pl.Categorical
    })

L’option `dtypes` est ici nécessaire sous peine d’une erreur qu’on ne
rencontrait pas avec `Pandas` car `Polars` est plus vigilant sur la
nature piégeuse de certaines variables comme les numéros de départements
français (qui sont tous numériques sauf les numéros Corse *“2A”* et
*“2B”*) et demande à l’utilisateur de données de fixer le type de ces
variables.

<div class="alert alert-info" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-comment"></i> Note</h3>

Pour les utilisateurs du [SSP Cloud](https://www.sspcloud.fr/), il
existe une méthode plus universelle pour récupérer des données
disponibles depuis le système de stockage. La librairie `S3FS` permet de
traiter le système de stockage S3 comme un *filesystem* (FS) local. Il
est donc possible de directement lire depuis un CSV en passant par
l’espace de stockage de la plateforme:

``` python
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "donnees-insee/diffusion/BPE/2019"

with fs.open(f"{BUCKET}/BPE_ENS.csv") as bpe_csv:
    df_bpe = pl.read_csv(
        bpe_csv, sep = ";",
        dtypes={
            "DEP": pl.Categorical,
            "DEPCOM": pl.Categorical
            })
    print(df_bpe.head())
```

    shape: (5, 7)
    ┌─────┬─────┬────────┬────────┬──────┬────────┬──────────┐
    │ REG ┆ DEP ┆ DEPCOM ┆ DCIRIS ┆ AN   ┆ TYPEQU ┆ NB_EQUIP │
    │ --- ┆ --- ┆ ---    ┆ ---    ┆ ---  ┆ ---    ┆ ---      │
    │ i64 ┆ cat ┆ cat    ┆ str    ┆ i64  ┆ str    ┆ i64      │
    ╞═════╪═════╪════════╪════════╪══════╪════════╪══════════╡
    │ 84  ┆ 01  ┆ 01001  ┆ 01001  ┆ 2019 ┆ A401   ┆ 2        │
    │ 84  ┆ 01  ┆ 01001  ┆ 01001  ┆ 2019 ┆ A404   ┆ 4        │
    │ 84  ┆ 01  ┆ 01001  ┆ 01001  ┆ 2019 ┆ A405   ┆ 2        │
    │ 84  ┆ 01  ┆ 01001  ┆ 01001  ┆ 2019 ┆ A504   ┆ 1        │
    │ 84  ┆ 01  ┆ 01001  ┆ 01001  ┆ 2019 ┆ A507   ┆ 1        │
    └─────┴─────┴────────┴────────┴──────┴────────┴──────────┘

</div>

# Comment utiliser Polars ?

Dans la suite de ce tutoriel, on va privilégier l’import depuis un
fichier `Parquet`, plus performant et plus fiable grâce au typage des
colonnes.

In [11]:
df_bpe = pl.read_parquet("bpe.parquet")

A l’instar d’autres outils modernes d’exploitation des données, `Polars`
expose un modèle de traitement basé sur des fonctions de haut niveau,
comme `select`, `filter` ou `groupby`, qui empruntent au langage SQL une
logique expressive du *“quoi ?”* plutôt que du *“comment ?”*.

La différence avec un package comme `Pandas` ou `dplyr` (`R`) est que
`Polars` repose explicitement sur le principe de l’exécution retardée
(*lazy evaluation*) qui consiste à optimiser le plan d’exécution fourni
par l’utilisateur de données pour gagner en performance.

Dans l’exemple qui suit, on commence par déclarer une exécution retardée
(via `lazy()`) qui va permettre au moteur sous-jacent d’optimiser le
traitement complet. Puis on exprime à l’aide des fonctions de haut
niveau ce que l’on veut faire :

1.  Filtrer le jeu de données pour ne garder les lignes pour lesquelles
    la colonne `TYPEQU` vaut `B316` (les stations-services)
2.  On regroupe au niveau département
3.  On compte le nombre d’occurrences pour chaque département via `agg`
4.  Le dernier appel - `collect()` - indique que le traitement peut être
    lancé (et donc optimisé et parallelisé par `Polars`).

In [12]:
df_stations_service = df_bpe.lazy().filter( # 1.
    pl.col("TYPEQU") == "B316"
).groupby( # 2.
    "DEP"
).agg( # 3.
    pl.count().alias("NB_STATION_SERVICE")
).collect() # 4.

df_stations_service.head(5)

DEP,NB_STATION_SERVICE
str,u32
"""62""",185
"""15""",32
"""48""",24
"""79""",55
"""81""",49


Du point de vue du code, la complexité induite par l’approche
d’évaluation retardée est donc légère (un `lazy` en début de plan et un
`collect` en fin de plan). Cependant, du point de vue de la performance,
cela va amener énormément de gain car cela permettra à `Polars`
d’optimiser l’ordre des opérations lorsque cela est possible pour gagner
en performance. Par exemple, si une étape comme `filter` arrive
tardivement, `Polars` peut être tenté de la faire intervenir plus tôt
que l’étape à laquelle elle a été déclarée si c’est possible puisque
traiter un *dataframe* allégé peut faire gagner beaucoup de temps.

## Lazy or not lazy ?

Les exemples qui suivent n’utilisent pas systématiquement la fonction
`lazy()` avant les opérations sur le *dataframe* pour montrer
l’alternative qui s’offre à l’utilisateur de `Polars`.

Cependant, l’usage de cette fonction est recommandée en général et en
particulier pour des traiements sur des tables volumineuses : en effet,
le mode *lazy* permet au moteur sous-jacent d’optimiser les traitements
et donc de gagner en rapidité (par exemple en parallélisant les
calculs).

## Les expressions

Les
[**expressions**](https://pola-rs.github.io/polars-book/user-guide/dsl/expressions.html)
sont au coeur des transformations de données en `Polars`. A partir de
l’exemple ci-dessous, on peut comprendre la nature d’une expression :

    pl.col("A").sort().head(2)

-   `pl` est l’alias pour le module `Polars` ;
-   `col("A")` pointe vers la colonne *“A”* ;
-   `sort()` est une fonction de tri à appliquer sur la colonne ;
-   l’opération `head` va permettre de matérialiser l’expression, ici
    pour sélectionner les deux premières lignes

Ici, nous avons utilisé `head` pour matérialiser l’expression. Le reste
de l’expression (les transformations antérieures) sera exécuté
lorsqu’une action comme `head` est à mettre en oeuvre.

Les actions qui déclenchent des opérations sont les suivantes:

-   l’aggrégation (`df.groupby(..).agg(..)`)
-   l’affichage de valeurs (`df.head(...)`)
-   l’appel explicite à une exécution (`df.collect()`)

Alors que les transformations suivantes ne génèrent pas d’exécution
immédiate :

-   la sélection d’observations (`df.filter(..)`)
-   la création de nouvelles variables (`df.with_columns(..)`) ou la
    sélection de colonnes (`df.select(...)`)

Au contraire, leur exécution est retardée à une exécution ultérieure.

# Sélection de données

Deux types de sélections sont possibles :

1.  une **sélection de variables** (en colonne), avec `select`
2.  une **sélection d’observations** (en ligne), avec `filter`

La combinaison des deux se faisant en chaînant l’appel à ces deux
fonctions.

## Sélection de variables

Commençons par sélectionner des variables en utilisant leurs noms :

In [13]:
df_bpe.select(
    ["DEPCOM", "TYPEQU", "NB_EQUIP"]
).head(5)

DEPCOM,TYPEQU,NB_EQUIP
str,str,str
"""01001""","""A129""","""1"""
"""01001""","""A401""","""2"""
"""01001""","""A402""","""1"""
"""01001""","""A404""","""2"""
"""01001""","""A405""","""2"""


Bien que cette méthode ne soit pas conseillée, il est bon de noter qu’il
est également possible de sélectionner via les positions des colonnes,
comme le permet `Pandas` :

In [14]:
df_bpe[:, 1:5].head(5)

AN,BV2012,DEP,DEPCOM
str,str,str,str
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""


On peut également s’appuyer sur des motifs de sélection des noms de
colonnes mobilisant des expressions régulières (ici `^DEP.*$` signifiant
*“débute par DEP”*):

In [15]:
df_bpe.select(
    pl.col("^DEP.*$")
).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


La fonction `select` acceptant des `list` Python, on peut construire des
sélecteurs assez puissants :

In [16]:
dep_cols = [cols for cols in df_bpe.columns if cols.startswith("DEP")] 

df_bpe.select(dep_cols).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


## Sélection d’observations

La sélection d’observations (de lignes) se fera grâce à la fonction
`filter`.

In [17]:
df_bpe.filter(
    (pl.col("DEPCOM") == "75101") &
    (pl.col("TYPEQU") == "B304")
).head(5)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010101""","""11""","""B3""","""B304""","""00851""","""2"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010201""","""11""","""B3""","""B304""","""00851""","""17"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010202""","""11""","""B3""","""B304""","""00851""","""3"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010203""","""11""","""B3""","""B304""","""00851""","""6"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010204""","""11""","""B3""","""B304""","""00851""","""7"""


De nombreux opérateurs sont disponibles pour faciliter cette sélection
d’observations. Ici, deux exemples :

-   `is_in` qui permet de filtrer sur un ensemble de valeurs ;
-   `is_null` pour trouver les données manquantes de la variable
    *“EPCI”*.

Par exemple, pour sélectionner les départements 75 et 92:

In [18]:
# Départements 75 ou 92
df_bpe.filter(
    pl.col("DEP").is_in(["75", "92"])
).head(5)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010201""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010301""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010402""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010102""","""11""","""A1""","""A105""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010402""","""11""","""A1""","""A120""","""00851""","""1"""


Pour sélectionner les observations où la variable EPCI a une valeur
manquante:

In [19]:
# Valeurs manquantes pour la variable EPCI
df_bpe.filter(
    pl.col("EPCI").is_null()
)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str


A l’inverse, si on désire ne sélectionner que les observations où la
colonne *“EPCI”* n’est pas manquante :

In [20]:
df_bpe.filter(
    pl.col("EPCI").is_not_null()
)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A5""","""A504""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A5""","""A505""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A5""","""A507""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""B""","""200069193""","""010010000""","""84""","""B2""","""B203""","""CSZ""","""1"""


Comme avec `Pandas`, il existe énormément de méthodes pratiques pour la
manipulation de données

## Renommage de variables

La fonction `rename` permet de lister les colonnes à renommer via un
dictionnaire Python :

In [21]:
df_bpe.rename({
    "DEPCOM" : "code_commune"
}).head(5)

AAV2020,AN,BV2012,DEP,code_commune,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


<div class="alert alert-info" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-comment"></i> Note</h3>

Petit piège, la logique est l’inverse de celle de `dplyr` : le
dictionnaire est sous la forme
`{"ancienne_colonne": "nouvelle_colonne"}`

</div>

Comme vu plus haut, construire des expressions de renommage plus
complexes pourra se faire en pur `Python` :

In [22]:
cols_minuscules = {cols: cols.lower() for cols in df_bpe.columns}

df_bpe.rename(cols_minuscules).head(5)

aav2020,an,bv2012,dep,depcom,dom,epci,dciris,reg,sdom,typequ,uu2020,nb_equip
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


## Trier une table

La fonction `sort` permet de trier la table sur une ou plusieurs
variables.

In [23]:
df_bpe.lazy()\
    .sort("DEPCOM", "TYPEQU")\
    .head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


L’ordre de tri - croissant par défaut - peut-être précisé pour chaque
variable.

In [24]:
df_bpe.lazy()\
    .sort("DEPCOM", "TYPEQU", descending=[True, False])\
    .head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A1""","""A128""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A1""","""A129""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A203""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A205""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A206""","""9F304""","""1"""


## Création de nouvelles variables

La création de nouvelles variables se fera via `with_columns`. Dans
l’exemple qui suit :

-   on convertit en entier numérique la variable `NB_EQUIP`
-   on calcule la somme cumulée (avec `cumsum`)
-   on nomme la nouvelle colonne `NB_EQUIP_SUM`

In [25]:
df_bpe.lazy()\
    .with_columns(pl.col('NB_EQUIP').cast(pl.Int64, strict=False))\
    .with_columns(
        pl.col("NB_EQUIP").cumsum().alias("NB_EQUIP_SUM"),
        pl.when(pl.col("NB_EQUIP") > 3).then(True).otherwise(False).alias("NB_EQUIP_3PLUS")
    ).head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP,NB_EQUIP_SUM,NB_EQUIP_3PLUS
str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,bool
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""",1,1,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""",2,3,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""",1,4,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""",2,6,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""",2,8,false


## Production de synthèses et d’agrégats

On peut produire des statistiques synthétiques sur notre jeu de données
avec la fonction `select`.

In [26]:
df_bpe.lazy()\
    .with_columns(pl.col('NB_EQUIP').cast(pl.Int64, strict=False))\
    .select(
        pl.col("NB_EQUIP").sum().alias("NB_EQUIP_TOT")
    ).head(5).collect()

NB_EQUIP_TOT
i64
2399536


Les fonctions `describe` et `glimpse` proposent un aperçu synthétique de
la table:

In [27]:
df_bpe.describe()

describe,AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""001""","""2021""","""01004""","""01""","""01001""","""A""","""200000172""","""010010000""","""01""","""A1""","""A101""","""00151""","""1"""
"""max""","""SAR""","""2021""","""CSZ""","""976""","""97617""","""G""","""CSZ""","""97617_IND""","""94""","""G1""","""G104""","""CSZ""","""99"""
"""median""",null,null,null,null,null,null,null,null,null,null,null,null,null


In [28]:
df_bpe.glimpse()

Rows: 1056914
Columns: 13
$ AAV2020  <str> 524, 524, 524, 524, 524, 524, 524, 524, 524, 524
$ AN       <str> 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021
$ BV2012   <str> 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093
$ DEP      <str> 01, 01, 01, 01, 01, 01, 01, 01, 01, 01
$ DEPCOM   <str> 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001
$ DOM      <str> A, A, A, A, A, A, A, A, B, C
$ EPCI     <str> 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193
$ DCIRIS   <str> 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000
$ REG      <str> 84, 84, 84, 84, 84, 84, 84, 84, 84, 84
$ SDOM     <str> A1, A4, A4, A4, A4, A5, A5, A5, B2, C1
$ TYPEQU   <str> A129, A401, A402, A404, A405, A504, A505, A507, B203, C104
$ UU2020   <str> CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ
$ NB_EQUIP <str> 1, 2, 1, 2, 2, 1, 1, 2, 1, 1


Les fonctions `groupby` et `agg` sont mobilisées pour créer des
aggrégats. Dans l’exemple ci-dessous :

1.  On précise que l’on veut grouper au niveau départemental
    (`groupby("DEP")`) ;
2.  On somme les occurences d’équipement de type boulangerie
    (`pl.col("TYPEQU") == "B203").sum()`) ;
3.  On donne un nom à la colonne portant la somme
    (`alias("NB_BOULANGERIES_TOT")`)

In [29]:
df_bpe.lazy().groupby("DEP").agg(
    (pl.col("TYPEQU") == "B203").sum().alias("NB_BOULANGERIES_TOT")
).sort("NB_BOULANGERIES_TOT", descending=True).head(5).collect()

DEP,NB_BOULANGERIES_TOT
str,u32
"""75""",731
"""59""",731
"""13""",576
"""69""",507
"""62""",440


# Explorer le plan d’exécution

Il peut être utile pour comprendre les opérations effectuées par
`Polars` de jeter un oeil au plan d’exécution.

Pour cela, il suffit pour cela de ne pas effectuer d’action comme
`collect` ou `head`.

In [30]:
df_bpe.lazy()\
    .with_columns(pl.col('NB_EQUIP').cast(pl.Int64, strict=False))\
    .select(
        pl.col("NB_EQUIP").sum().alias("NB_EQUIP_TOT")
    )

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan) 
 
 SELECT [col("NB_EQUIP").sum().alias("NB_EQUIP_TOT")] FROM WITH_COLUMNS: [col("NB_EQUIP").cast(Int64)] DF ["AAV2020", "AN", "BV2012", "DEP"]; PROJECT */13 COLUMNS; SELECTION: "None"

<div class="alert alert-info" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-comment"></i> Note</h3>

L’opération d’affichage du plan est ici instantanée puisqu’aucune action
n’a été effectuée, seulement des transformations dont l’activation est
retardée en attente d’une action n’ayant pas lieu.

</div>